In [2]:
import csv
import cv2
import numpy as np

lines = []
with open("../sdc_training_data/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    sourcepath = line[0]
    filename = sourcepath.split('/')[-1]
    currentpath = '../sdc_training_data/IMG/' + filename
    image = cv2.imread(currentpath)
    images.append(image)
    
    measurement = float(line[3])
    measurements.append(measurement)
    
augmeted_images, augmeted_measurements = [], []
for image, measurement in zip(images, measurements):
    augmeted_images.append(cv2.flip(image, 1))
    augmeted_measurements.append(-measurement)
    augmeted_images.append(image)
    augmeted_measurements.append(measurement)
    
X_train = np.array(augmeted_images)
y_train = np.array(augmeted_measurements)

from keras.models import Sequential
from keras.layers import Input, Convolution2D, MaxPooling2D, Flatten, Dense, Lambda


model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
#model.add(Flatten(input_shape = (160, 320, 3)))

model.add(Convolution2D(6,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, validation_split=0.3, shuffle=True, nb_epoch=7)

model.save("model.h5")


Using TensorFlow backend.


Train on 2507 samples, validate on 1075 samples
Epoch 1/7
2507/2507 [==============================] - 191s - loss: 0.8461 - val_loss: 0.0140